# Выбор локации для скважины

Вы работаете в компании «ГлавРосГосНефть» и стоит задача выбрать месторасположение для бурения новой нефтяной скважины. У вас есть данные о пробах нефти из трёх регионов, каждый из которых содержит информацию о 10 000 месторождениях, включая качество нефти и объём её запасов. Ваша цель - построить модель машинного обучения, которая поможет определить регион, где добыча нефти принесёт наибольшую прибыль. Для этого вы планируете использовать метод Bootstrap для оценки возможной прибыли и рисков. 

Процесс выбора локации будет следующим:

1. Изучение признаков месторождений в каждом регионе.
2. Построение модели для оценки объёма запасов нефти.
3. Определение месторождений с наивысшими оценками значений, исходя из бюджета компании и стоимости разработки одной скважины.
4. Оценка прибыли, которая будет равна сумме прибыли от выбранных месторождений.

## Загрузка и подготовка данных

### Импорт библиотек и глобальных

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import sqrt

state = np.random.RandomState(12345)

# Первичный обзор данных

In [6]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

In [7]:
print(f'''##### Data_0: #####
            {data_0}''')
print()
print(f'''##### Data_1: #####
            {data_1}''')
print()
print(f'''##### Data_2: #####
            {data_2}''')
print()

##### Data_0: #####
                      id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns]

##### Data_1: #####
                      id         f0         f1        f2     product
0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
1      62mP7  14.272088  -3.475083  0.999183   26.953261
2      vyE1P   6.263187  -5.948386  5.001160  134.766305
3      KcrkZ -13.081

In [8]:
print(f'''##### Data_0: #####
            {data_0.info()}''')
print()
print(f'''##### Data_1: #####
            {data_1.info()}''')
print()
print(f'''##### Data_2: #####
            {data_2.info()}''')
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
##### Data_0: #####
            None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
##### Data_1: #####
            None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000

По всем датасетам можно сделать вывод, что пропусков нет, типы данных соответствуют действительности, имеем 100,000 строк и 5 колонов

Набор данных содержит в себе следующие столбцы:

`id` - уникальный идентификатор скважины;

`f0, f1, f2` -  три признака точек (неважно, что они означают, но сами признаки значимы);

`product` - объём запасов в скважине (тыс. баррелей).


In [9]:
print(f'''##### Data_0: #####
            {data_0.describe()}''')
print()
print(f'''##### Data_1: #####
            {data_1.describe()}''')
print()
print(f'''##### Data_2: #####
            {data_2.describe()}''')
print()

##### Data_0: #####
                              f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

##### Data_1: #####
                              f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        

По характеристическим данным можно сделать вывод, что для первого региона средний обьем запасов в разных места равен 92 тысячам барелей, максимальное 185 тысяч, 
Для второго региона среднее 68 тысяч, максимальное 137 тысяч
Для третьего региона среднее 95 тысяч, максимальное 190 тысяч, что является лучшим результатом по объёму запасов

In [14]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [15]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [16]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Далее построим матрицы корреляций

In [21]:
data_0.corr()

/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/474601201.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_0.corr()


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [22]:
data_1.corr()

/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/845247271.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_1.corr()


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [23]:
data_2.corr()

/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/1391761772.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_2.corr()


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


### Проверка дубликатов

In [28]:
print('data_0: ', data_0.duplicated().sum())
print('data_1: ', data_1.duplicated().sum())
print('data_2: ', data_2.duplicated().sum())

data_0:  0
data_1:  0
data_2:  0


**Вывод:**

1. пропущенные значения отсутствуют
2. столбцы названы корректно, переименовывание не требуется
3. столбец "id" - для обучения не нужен
4. features:
    * f0
    * f1
    * f2
5. target:
    * product
6. дубликаты в данных отсутствуют 

## Подготовка данных

In [31]:
features_0 = data_0[['f0', 'f1', 'f2']]
target_0 = data_0['product']

In [32]:
features_1 = data_1[['f0', 'f1', 'f2']]
target_1 = data_1['product']

In [33]:
features_2 = data_2[['f0', 'f1', 'f2']]
target_2 = data_2['product']

**Вывод:**
1. Сформированы наборы features:

    * features_0
    * features_1
    * features_2
    
    
2. Сформированы наборы target:

    * target_0
    * target_1
    * target_2

## Обучение и проверка модели

### Создание обучающих/валидационных наборов

**Функция создания обучающих/валидационных наборов**

In [34]:
def divide(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = state)
    return features_train, target_train, features_valid, target_valid
features_train_0, target_train_0, features_valid_0, target_valid_0 = divide(features_0, target_0)
features_train_1, target_train_1, features_valid_1, target_valid_1 = divide(features_1, target_1)
features_train_2, target_train_2, features_valid_2, target_valid_2 = divide(features_2, target_2)

**Создание обучающих/валидационных наборов для первого региона**

In [35]:
print(f'''##### Feature_0_train shape: #####
            {features_train_0.shape}''')
print()
print(f'''##### Feature_0_valid shape: #####
            {features_valid_0.shape}''')
print()
print(f'''##### Target_0_train shape: #####
            {target_train_0.shape}''')
print()
print(f'''##### Target_0_valid shape: #####
            {target_valid_0.shape}''')
print()
print(f'''##### Feature_1_train shape: #####
            {features_train_1.shape}''')
print()
print(f'''##### Feature_1_valid shape: #####
            {features_valid_1.shape}''')
print()
print(f'''##### Target_1_train shape: #####
            {target_train_1.shape}''')
print()
print(f'''##### Target_1_valid shape: #####
            {target_valid_1.shape}''')

##### Feature_0_train shape: #####
            (75000, 3)

##### Feature_0_valid shape: #####
            (25000, 3)

##### Target_0_train shape: #####
            (75000,)

##### Target_0_valid shape: #####
            (25000,)

##### Feature_1_train shape: #####
            (75000, 3)

##### Feature_1_valid shape: #####
            (25000, 3)

##### Target_1_train shape: #####
            (75000,)

##### Target_1_valid shape: #####
            (25000,)


**Вывод:**

1. Созданы обучающие наборы:

    * features_0_train
    * target_0_train
    * features_1_train
    * target_1_train
    * features_2_train
    * target_2_train
    
    
2. Созданы валидационные наборы:

    * features_0_valid
    * target_0_valid
    * features_1_valid
    * target_1_valid
    * features_2_valid
    * target_2_valid

### Обучение и проверка моделей

**Функция обучения/предсказания моделей:**

In [36]:
def model_predict(features_train, target_train, features_valid, target_valid):
    scaler = StandardScaler()
    scaler.fit(features_train)
    scaler.transform(features_train)
    scaler.transform(features_valid)
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    return model.score(features_valid, target_valid), predicted_valid

**Создание обучающих/валидационных наборов для первого региона**

In [37]:
model_0_score, predicted_valid_0 = model_predict(features_train = features_train_0, 
                                                 target_train = target_train_0, 
                                                 features_valid = features_valid_0, 
                                                 target_valid = target_valid_0)

**Создание обучающих/валидационных наборов для второго региона**

In [38]:
model_1_score, predicted_valid_1 = model_predict(features_train = features_train_1, 
                                                 target_train = target_train_1, 
                                                 features_valid = features_valid_1, 
                                                 target_valid = target_valid_1)

**Создание обучающих/валидационных наборов для третьего региона**

In [39]:
model_2_score, predicted_valid_2 = model_predict(features_train = features_train_2, 
                                                 target_train = target_train_2, 
                                                 features_valid = features_valid_2, 
                                                 target_valid = target_valid_2)

### Результаты предсказаний для всех регионов

In [40]:
target_0_predict = pd.concat([target_valid_0.reset_index(drop=True), pd.DataFrame(predicted_valid_0)], axis=1)
target_1_predict = pd.concat([target_valid_1.reset_index(drop=True), pd.DataFrame(predicted_valid_1)], axis=1)
target_2_predict = pd.concat([target_valid_2.reset_index(drop=True), pd.DataFrame(predicted_valid_2)], axis=1)

target_0_predict = target_0_predict.rename(columns={0:'predicted_product'})
target_1_predict = target_1_predict.rename(columns={0:'predicted_product'})
target_2_predict = target_2_predict.rename(columns={0:'predicted_product'})

In [42]:
model_results = pd.DataFrame({'real_materials_volume'        :[target_valid_0.sum(),
                                                               target_valid_1.sum(),
                                                               target_valid_2.sum()],
                              'predict_material_volume'      : [predicted_valid_0.sum(),
                                                                predicted_valid_1.sum(),
                                                                predicted_valid_2.sum()],
                              'mean_predict_material_volume' : [predicted_valid_0.mean(),
                                                                predicted_valid_1.mean(),
                                                                predicted_valid_2.mean()],
                              'RMSE'                         : [sqrt(mean_squared_error(target_valid_0, predicted_valid_0)),
                                                                sqrt(mean_squared_error(target_valid_1, predicted_valid_1)),
                                                                sqrt(mean_squared_error(target_valid_2, predicted_valid_2))],
                              'R2-score'                     : [model_0_score,
                                                                model_1_score,
                                                                model_2_score]
}
)

model_results

,real_materials_volume,predict_material_volume,mean_predict_material_volume,RMSE,R2-score
0,2.301965e+06,2.314814e+06,92.592568,37.579422,0.279943
1,1.719291e+06,1.719249e+06,68.769951,0.889737,0.999626
2,2.368724e+06,2.377188e+06,95.087528,39.958042,0.202615


**Вывод:**

1. Правильные ответы и предсказания моделей сохранены в:
    * model_results
    
    
2. Анализ характеристик каждого региона показал:
    * наиболее перспектиные регионы по среднему предсказанному запасу сырья являются 1-й и 3-й регион
    * минимальное значение RMSE получено для 2-го региона
    * максимальное качество(коэффициент детерминации) имеет модель полученная для 2-го региона

## Подготовка к расчёту прибыли

In [43]:
MATERIAL_UNIT_PRICE = 10**3*450                      # доход с единицы сырья(1000 баррелей)
INVESTMENT_SUM = 10**10                             # бюджет на разработку скважин
HOLE_PRICE = INVESTMENT_SUM/200                      # стоимость стротельства одной скважины
HOLE_TRESHOLD = HOLE_PRICE/MATERIAL_UNIT_PRICE       # необходимый запас сырья для безубыточности скважины
REGION_TRESHOLD = INVESTMENT_SUM/MATERIAL_UNIT_PRICE # необходимый запас сырья для безубыточности региона
print(f'''
Порог безубыточности скважины: {HOLE_TRESHOLD} ед. сырья
Порог безубыточности региона:  {REGION_TRESHOLD} ед. сырья''')


Порог безубыточности скважины: 111.11111111111111 ед. сырья
Порог безубыточности региона:  22222.222222222223 ед. сырья


**Вывод:**

Для начала освоения региона необходимо, чтобы запасы сырья в отобранных 200 скважинах были более 22223 ед. сырья. Запасы сырьяя в каждой отобранной скважине должны быть более 112 ед. сырья.

## Расчёт прибыли и рисков 

### Функция расчета прибыли

In [44]:
def material_sum_revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    material_sum = selected.sum()
    material_revenue = material_sum*MATERIAL_UNIT_PRICE - INVESTMENT_SUM
    return pd.DataFrame({'material_sum':[material_sum], 'material_revenue':[material_revenue]})

### Функция bootstrap

In [45]:
def bootstrap(target_valid, probabilities_valid, hole_count):
    region_boot = pd.DataFrame()
    for i in range(1000):
        target_subsample = target_valid.sample(n=hole_count, replace=True, random_state=state)
        probs_subsample = probabilities_valid[target_subsample.index]
        region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
    return  region_boot

### Распределение значений запасов сырья и прибыли для регионов

In [46]:
region_0_boot = bootstrap(target_valid=target_0_predict['product'],
                          probabilities_valid=target_0_predict['predicted_product'],
                          hole_count=500)

region_1_boot = bootstrap(target_valid=target_1_predict['product'],
                            probabilities_valid=target_1_predict['predicted_product'],
                            hole_count=500)

region_2_boot = bootstrap(target_valid=target_2_predict['product'],
                            probabilities_valid=target_2_predict['predicted_product'],
                            hole_count=500)

/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/1174849380.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/1174849380.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/4l/7yb4w59x1k3bhq8mbgbzjry00000gn/T/ipykernel_15327/1174849380.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

In [47]:
region_0_boot

,material_sum,material_revenue
0,23123.568067,4.056056e+08
1,23328.235012,4.977058e+08
2,23234.513019,4.555309e+08
3,22668.387302,2.007743e+08
4,23180.189675,4.310854e+08
...,...,...
995,23093.069506,3.918813e+08
996,23043.832616,3.697247e+08
997,23198.583886,4.393627e+08
998,23107.863816,3.985387e+08


### Функция расчета доверительного интервала 

In [48]:
def get_confidence_interval(region_boot):
    return (region_boot.material_revenue.quantile(0.025), region_boot.material_revenue.quantile(0.975))

### Расчет доверительного интервала

In [49]:
confidence_interval_region_0 = get_confidence_interval(region_0_boot)
confidence_interval_region_1 = get_confidence_interval(region_1_boot)
confidence_interval_region_2 = get_confidence_interval(region_2_boot)

### Функция расчета риска разработки региона

In [50]:
def risk_percent(region_boot):
    low_treshold_count = len(region_boot.query('material_revenue <= 0'))
    return low_treshold_count/len(region_boot) * 100

### Расчет риска разработки региона

In [51]:
risk_region_0 = risk_percent(region_0_boot)
risk_region_1 = risk_percent(region_1_boot)
risk_region_2 = risk_percent(region_2_boot)

## Формирование итоговой таблицы

In [52]:
region_final = pd.DataFrame({'Средняя прибыль региона, руб.'      : [region_0_boot.material_revenue.mean(),
                                                                     region_1_boot.material_revenue.mean(),
                                                                     region_2_boot.material_revenue.mean()],
                             '95%-й доверительный интервал, руб.' : [confidence_interval_region_0,
                                                                     confidence_interval_region_1,
                                                                     confidence_interval_region_2],
                             'Вероятность убытков, %'             : [risk_region_0,
                                                                     risk_region_1,
                                                                     risk_region_2]}, index=['Регион 1','Регион 2','Регион 3'])
region_final


,"Средняя прибыль региона, руб.","95%-й доверительный интервал, руб.","Вероятность убытков, %"
Регион 1,4.238972e+08,"(-76187813.89036272, 957846531.951783)",4.8
Регион 2,5.132567e+08,"(108066895.23396212, 928574439.2324963)",0.6
Регион 3,3.811204e+08,"(-142800630.08786878, 893380565.7504003)",7.4


# Вывод:

**Для разработки рекомендуется "Регион 2", по следующим причинам:**
   * минимальная вероятность убытков ~ 0.4%
   * максимальная средняя прибыль из предложенных для анализа регионов